In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image

def create_mask(image_size, bbox):

    mask = np.zeros(image_size, dtype=np.uint8)
    x_min, y_min, x_max, y_max = bbox
    mask[y_min:y_max, x_min:x_max] = 1
    return mask

# Example usage:
image_size = (100, 100) # replace with your image size
bbox = (10, 20, 30, 40) # replace with your bounding box coordinates
mask = create_mask(image_size, bbox)

# Convert to a TensorFlow tensor
mask_tensor = tf.convert_to_tensor(mask, dtype=tf.uint8)

# Optionally, save the mask as an image for visualization
mask_image = Image.fromarray(mask * 255) # multiply by 255 to make the mask visible
mask_image.save('mask.png')


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Load a pre-trained object detection model from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")

# Define the function
def object_detection_and_masking_function(image):
    # Convert the image to a tensor and add a batch dimension
    image_tensor = tf.convert_to_tensor(image)
    image_tensor = image_tensor[tf.newaxis, ...]

    # Perform detection
    detector_output = detector(image_tensor)

    # Process the output to extract bounding boxes and class labels
    detection_boxes = detector_output['detection_boxes'][0].numpy()
    detection_scores = detector_output['detection_scores'][0].numpy()
    detection_classes = detector_output['detection_classes'][0].numpy()

    # Confidence threshold for detection
    confidence_threshold = 0.5

    # List to hold masks and annotations
    masks = []
    annotations = []

    # Go through the detections and create masks for high confidence detections
    for i in range(detection_boxes.shape[0]):
        if detection_scores[i] > confidence_threshold:
            # Get the bounding box coordinates
            y_min, x_min, y_max, x_max = detection_boxes[i]
            y_min, x_min, y_max, x_max = int(y_min * image.shape[0]), int(x_min * image.shape[1]), \
                                          int(y_max * image.shape[0]), int(x_max * image.shape[1])

            # Create a mask for the bounding box
            # Note that this is a simple binary mask for a rectangle
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            mask[y_min:y_max, x_min:x_max] = 1

            # Append to lists
            masks.append(mask)
            annotations.append({
                'box': [x_min, y_min, x_max, y_max],
                'class': detection_classes[i],
                'score': detection_scores[i]
            })

    return masks, annotations

# To use this function, pass an image as a numpy array:
#image_np = np.array(Image.open('/content/Unknown.jpeg'))
#masks, annotations = object_detection_and_masking_function(image_np)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load an image
image_path = '/content/Unknown2.jpeg'
image_np = np.array(Image.open(image_path))

# Get masks and annotations
masks, annotations = object_detection_and_masking_function(image_np)

# Function to display images, masks, and annotations
def display_images_masks_annotations(image, masks, annotations):
    plt.figure(figsize=(15, 10))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Original Image\nCredit: Bytes of Intelligence')

    plt.subplot(1, 3, 2)
    plt.imshow(image)
    plt.imshow(masks[0], cmap='jet', alpha=0.5)  # Change masks[0] to iterate over all masks if multiple objects are detected
    plt.title('Image with Mask\nCredit: Bytes of Intelligence')

    plt.subplot(1, 3, 3)
    for annotation in annotations:
        x_min, y_min, x_max, y_max = annotation['box']
        plt.plot([x_min, x_min, x_max, x_max, x_min], [y_min, y_max, y_max, y_min, y_min], 'r-')
    plt.imshow(image)
    plt.title('Image with Bounding Boxes\nCredit: Bytes of Intelligence')

    plt.show()

# Display the results
display_images_masks_annotations(image_np, masks, annotations)


In [ ]:
def batch_object_detection_and_masking_function(images, model, threshold=0.5):
    # Stack images into one large batch
    batched_images = tf.stack(images, axis=0)

    # Perform detection on the entire batch
    detector_output = model(batched_images)

    # Initialize lists to hold the masks and annotations for each image
    batch_masks = []
    batch_annotations = []

    # Iterate over each image in the batch
    for i in range(batched_images.shape[0]):
        # Process the output to extract bounding boxes and class labels
        detection_boxes = detector_output['detection_boxes'][i].numpy()
        detection_scores = detector_output['detection_scores'][i].numpy()
        detection_classes = detector_output['detection_classes'][i].numpy()

        # Lists to hold masks and annotations for the current image
        masks = []
        annotations = []

        # Go through the detections and create masks for high confidence detections
        for j in range(detection_boxes.shape[0]):
            if detection_scores[j] > threshold:
                # Get the bounding box coordinates
                y_min, x_min, y_max, x_max = detection_boxes[j]
                y_min, x_min, y_max, x_max = int(y_min * images[i].shape[0]), int(x_min * images[i].shape[1]), \
                                              int(y_max * images[i].shape[0]), int(x_max * images[i].shape[1])

                # Create a mask for the bounding box
                mask = np.zeros(images[i].shape[:2], dtype=np.uint8)
                mask[y_min:y_max, x_min:x_max] = 1

                # Append to lists
                masks.append(mask)
                annotations.append({
                    'box': [x_min, y_min, x_max, y_max],
                    'class': detection_classes[j],
                    'score': detection_scores[j]
                })

        # Append masks and annotations for the current image to the batch lists
        batch_masks.append(masks)
        batch_annotations.append(annotations)

    return batch_masks, batch_annotations


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained object detection model from TensorFlow Hub
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"
model = hub.load(model_url)

# List of image paths
image_paths = ['/content/Unknown2.jpeg']

# The expected input size for the SSD MobileNet V2 model is 320x320
input_size = (320, 320)

# Initialize the list to hold the preprocessed images
images = []

for img_path in image_paths:
    # Load the image
    img = load_img(img_path, target_size=input_size)

    # Convert the image to a numpy array
    img_array = img_to_array(img)

    # Convert the image array to uint8
    img_array = np.uint8(img_array)

    # Append the processed image tensor to our list
    images.append(img_array)

# Convert the list of images to a batched tensor with the correct dtype
image_tensors = tf.convert_to_tensor(images, dtype=tf.uint8)

# Your object detection and masking function here...

# Get masks and annotations for the batch
batch_masks, batch_annotations = batch_object_detection_and_masking_function(image_tensors, model)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_detections(image, masks, annotations):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    # Draw each mask
    for mask in masks:
        masked_image = np.ma.masked_where(mask == 0, mask)
        ax.imshow(masked_image, 'jet', alpha=0.5)

    # Draw each bounding box
    for annotation in annotations:
        x_min, y_min, x_max, y_max = annotation['box']
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                 linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.axis('off')
    plt.show()

# Assuming you have at least one image in your batch
visualize_image = images[0]  # First image of the batch
visualize_masks = batch_masks[0]  # Masks for the first image
visualize_annotations = batch_annotations[0]  # Annotations for the first image

# Visualize the detections for the first image
visualize_detections(visualize_image, visualize_masks, visualize_annotations)
